# 🧠 Hybrid Brain Tumor Detection System
## BME 271D Final Project - Ege, Max, Sasha

### Combining Frequency-Domain and Spatial-Domain Analysis

**Our Approach:**
1. **FFT-Based Detection** - Analyzes texture and edges in frequency domain
2. **Blob Detection** - Identifies bright, circular masses in spatial domain  
3. **Ensemble Voting** - Combines both methods for robust detection

**Why This Works:**
- FFT excels at finding texture differences and boundaries
- Blob detection catches very bright, distinct masses
- Combined approach reduces false positives and negatives

In [ ]:
# ========== SETUP ==========
!pip install -q numpy matplotlib scipy scikit-image pandas

!wget -q https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/tumor_segmentation.py
!wget -q https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/generate_realistic_tumors.py

!mkdir -p data/images data/masks
!wget -q -P data/images/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/images/tumor_001.png
!wget -q -P data/images/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/images/tumor_002.png
!wget -q -P data/images/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/images/tumor_003.png
!wget -q -P data/masks/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/masks/tumor_001.png
!wget -q -P data/masks/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/masks/tumor_002.png
!wget -q -P data/masks/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/masks/tumor_003.png

import tumor_segmentation as ts
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image as PILImage
from google.colab import files
from scipy import ndimage
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops
from skimage.morphology import remove_small_objects, binary_closing, binary_opening, disk
from skimage.feature import blob_log

def load_image_safe(filepath):
    img = PILImage.open(filepath).convert('L')
    return np.array(img).astype(np.float64) / 255.0

def load_mask_safe(filepath):
    img = PILImage.open(filepath).convert('L')
    return (np.array(img) > 127).astype(np.uint8)

# ========== METHOD 1: FFT-BASED DETECTION ==========
def fft_tumor_detection(image, sensitivity=0.5):
    """
    Frequency-domain tumor detection using FFT filtering.
    Detects regions with abnormal texture/frequency content.
    """
    h, w = image.shape
    mean_val = image.mean()
    std_val = image.std()
    
    results = {}
    
    # High-pass: enhances edges and boundaries
    try:
        hp_img, _, _ = ts.filter_pipeline(image, 'hp', cutoff_radius=30)
        hp_img = (hp_img - hp_img.min()) / (hp_img.max() - hp_img.min() + 1e-8)
        
        hp_thresh = np.percentile(hp_img, 85 - sensitivity * 10)
        hp_mask = hp_img > hp_thresh
        
        # Must also be somewhat bright
        hp_mask = np.logical_and(hp_mask, image > mean_val)
        hp_mask = binary_closing(hp_mask, disk(3))
        hp_mask = remove_small_objects(hp_mask, min_size=50)
        
        results['FFT_HighPass'] = hp_mask
    except:
        results['FFT_HighPass'] = np.zeros_like(image, dtype=bool)
    
    # Band-pass: targets tumor-specific frequencies
    try:
        bp_img, _, _ = ts.filter_pipeline(image, 'bp', r1=10, r2=50)
        bp_img = (bp_img - bp_img.min()) / (bp_img.max() - bp_img.min() + 1e-8)
        
        bp_thresh = np.percentile(bp_img, 80 - sensitivity * 10)
        bp_mask = bp_img > bp_thresh
        
        bp_mask = np.logical_and(bp_mask, image > mean_val)
        bp_mask = binary_closing(bp_mask, disk(3))
        bp_mask = remove_small_objects(bp_mask, min_size=50)
        
        results['FFT_BandPass'] = bp_mask
    except:
        results['FFT_BandPass'] = np.zeros_like(image, dtype=bool)
    
    # Combined FFT: regions detected by both filters
    fft_combined = np.logical_or(results['FFT_HighPass'], results['FFT_BandPass'])
    results['FFT_Combined'] = fft_combined
    
    return results

# ========== METHOD 2: BLOB DETECTION ==========
def blob_tumor_detection(image, sensitivity=0.5):
    """
    Spatial-domain blob detection.
    Finds bright, circular masses using intensity and shape.
    """
    h, w = image.shape
    mean_val = image.mean()
    std_val = image.std()
    
    # Find VERY bright regions (tumors are usually bright white)
    bright_threshold = mean_val + (2.0 - sensitivity * 0.5) * std_val
    bright_mask = image > bright_threshold
    
    # Also use top percentile approach
    percentile_threshold = np.percentile(image, 90 - sensitivity * 5)
    percentile_mask = image > percentile_threshold
    
    # Combine both approaches
    blob_mask = np.logical_or(bright_mask, percentile_mask)
    
    # Clean up
    blob_mask = binary_opening(blob_mask, disk(2))
    blob_mask = binary_closing(blob_mask, disk(4))
    blob_mask = remove_small_objects(blob_mask, min_size=100)
    
    # Filter by shape: keep compact regions
    labeled = label(blob_mask)
    filtered = np.zeros_like(blob_mask)
    
    for region in regionprops(labeled):
        # Size check: not too small, not too big
        if region.area < 50 or region.area > h * w * 0.3:
            continue
        
        # Shape check: somewhat round
        circularity = 4 * np.pi * region.area / (region.perimeter ** 2 + 1e-8)
        if circularity < 0.1:  # Too elongated
            continue
        
        # Intensity check: significantly brighter than surroundings
        region_intensity = image[labeled == region.label].mean()
        if region_intensity < mean_val + 0.5 * std_val:
            continue
        
        filtered[labeled == region.label] = True
    
    return {'Blob_Detection': filtered}

# ========== METHOD 3: HYBRID ENSEMBLE ==========
def hybrid_tumor_detection(image, sensitivity=0.5):
    """
    Hybrid detection combining FFT and Blob methods.
    
    Voting system:
    - FFT methods get 2 votes total
    - Blob detection gets 1 vote
    - Need 2+ votes to declare tumor presence
    """
    # Run both detection methods
    fft_results = fft_tumor_detection(image, sensitivity)
    blob_results = blob_tumor_detection(image, sensitivity)
    
    # All results
    all_results = {**fft_results, **blob_results}
    
    # Voting: FFT Combined gets 1.5 votes, Blob gets 1 vote
    vote_map = (fft_results['FFT_Combined'].astype(float) * 1.5 +
                blob_results['Blob_Detection'].astype(float) * 1.0)
    
    # Need at least 1.5 votes (either FFT alone or Blob + partial FFT)
    combined_mask = vote_map >= (1.5 - sensitivity * 0.3)
    
    # Final cleanup
    combined_mask = binary_closing(combined_mask, disk(3))
    combined_mask = remove_small_objects(combined_mask, min_size=100)
    
    all_results['Hybrid_Combined'] = combined_mask
    
    return all_results

def analyze_detection(image, mask):
    """
    Analyze detection results and return metrics.
    """
    h, w = image.shape
    total_pixels = h * w
    tumor_pixels = mask.sum()
    tumor_area_pct = (tumor_pixels / total_pixels) * 100
    
    if tumor_pixels < 50:  # Very small detection
        return {'detected': False, 'area_percent': 0, 'center': None, 'confidence': 0}
    
    coords = np.where(mask)
    if len(coords[0]) == 0:
        return {'detected': False, 'area_percent': 0, 'center': None, 'confidence': 0}
    
    center = (int(np.mean(coords[0])), int(np.mean(coords[1])))
    
    labeled = label(mask)
    regions = regionprops(labeled, intensity_image=image)
    
    if len(regions) == 0:
        return {'detected': False, 'area_percent': 0, 'center': None, 'confidence': 0}
    
    largest = max(regions, key=lambda x: x.area)
    
    # Confidence scoring
    # Size: 0.5% - 15% is good
    if 0.5 <= tumor_area_pct <= 15:
        size_score = 1.0
    elif tumor_area_pct < 0.5:
        size_score = tumor_area_pct / 0.5
    else:
        size_score = max(0, 1 - (tumor_area_pct - 15) / 20)
    
    # Shape: circularity
    circularity = 4 * np.pi * largest.area / (largest.perimeter ** 2 + 1e-8)
    shape_score = min(circularity * 2, 1.0)
    
    # Intensity: tumor should be bright
    tumor_intensity = image[mask].mean()
    bg_intensity = image[~mask].mean() if (~mask).any() else 0
    contrast = tumor_intensity - bg_intensity
    contrast_score = min(max(contrast * 5, 0), 1.0)
    
    confidence = (size_score * 0.3 + shape_score * 0.3 + contrast_score * 0.4)
    
    # Minimum confidence threshold
    if confidence < 0.25:
        return {'detected': False, 'area_percent': 0, 'center': None, 'confidence': 0}
    
    return {
        'detected': True,
        'area_percent': tumor_area_pct,
        'center': center,
        'confidence': np.clip(confidence, 0, 1),
        'num_regions': len(regions)
    }

image = None
mask = None
print('✅ Hybrid Brain Tumor Detection System Ready!')
print('   Methods: FFT (High-pass + Band-pass) + Blob Detection')
print('   Ensemble voting for robust detection')

---
## 📁 Load Brain MRI

In [ ]:
# ========== UPLOAD YOUR BRAIN MRI ==========
print('Upload brain MRI scan (JPG, PNG, etc.):')

uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    image = load_image_safe(filename)
    mask = None
    
    print(f'\n✅ Loaded: {filename}')
    print(f'   Size: {image.shape[1]} x {image.shape[0]} pixels')
    print(f'   Intensity range: {image.min():.2f} - {image.max():.2f}')
    
    plt.figure(figsize=(8, 8))
    plt.imshow(image, cmap='gray')
    plt.title('Loaded Brain MRI', fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.show()
else:
    print('❌ No file uploaded')

---
## 🔬 Step 1: FFT Spectrum Analysis

In [ ]:
if image is not None:
    print('Computing FFT spectrum...')
    F_shift, mag = ts.compute_fft_spectrum(image)
    fig = ts.visualize_frequency_spectrum(image, F_shift)
    plt.show()
    
    print('\n📊 The FFT shows the frequency content of the brain image.')
    print('   Tumors often have different texture → different frequency signature!')
else:
    print('⚠️ No image loaded!')

---
## 🎯 Step 2: Run Hybrid Detection

This compares **three approaches:**
1. FFT-based methods (frequency domain)
2. Blob detection (spatial domain)
3. Hybrid combination (ensemble)

In [ ]:
if image is not None:
    print('Running hybrid tumor detection...\n')
    
    # Run detection
    results = hybrid_tumor_detection(image, sensitivity=0.5)
    
    # Methods to display
    methods = ['FFT_HighPass', 'FFT_BandPass', 'FFT_Combined', 'Blob_Detection', 'Hybrid_Combined']
    
    fig, axes = plt.subplots(2, 3, figsize=(16, 11))
    axes = axes.ravel()
    
    # Original
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title('Original Brain MRI', fontsize=13, fontweight='bold')
    axes[0].axis('off')
    
    # Each method
    for idx, method in enumerate(methods, 1):
        detected_mask = results[method]
        analysis = analyze_detection(image, detected_mask)
        
        axes[idx].imshow(image, cmap='gray')
        axes[idx].imshow(detected_mask, cmap='Reds', alpha=0.5)
        
        if analysis['center']:
            row, col = analysis['center']
            axes[idx].plot(col, row, 'g+', markersize=20, markeredgewidth=3)
        
        # Status
        if analysis['detected']:
            status = f"DETECTED\nArea: {analysis['area_percent']:.1f}%\nConf: {analysis['confidence']*100:.0f}%"
        else:
            status = "Not detected"
        
        axes[idx].set_title(f"{method}\n{status}", fontsize=11, fontweight='bold')
        axes[idx].axis('off')
    
    plt.suptitle('🧠 Hybrid Tumor Detection: Method Comparison', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Summary table
    print('\n' + '='*80)
    print(f'{"Method":<20} {"Type":<15} {"Detected":<12} {"Area %":<10} {"Confidence":<12}')
    print('='*80)
    
    method_types = {
        'FFT_HighPass': 'Frequency',
        'FFT_BandPass': 'Frequency',
        'FFT_Combined': 'Frequency',
        'Blob_Detection': 'Spatial',
        'Hybrid_Combined': '✨ HYBRID'
    }
    
    for method in methods:
        analysis = analyze_detection(image, results[method])
        det = '✓ YES' if analysis['detected'] else '✗ No'
        area = f"{analysis['area_percent']:.2f}%" if analysis['detected'] else '-'
        conf = f"{analysis['confidence']*100:.0f}%" if analysis['detected'] else '-'
        mtype = method_types[method]
        print(f'{method:<20} {mtype:<15} {det:<12} {area:<10} {conf:<12}')
    
    print('='*80)
else:
    print('⚠️ No image loaded!')

---
## 📋 Step 3: FINAL DIAGNOSIS

Using the **Hybrid Combined** method (best of both worlds)

In [ ]:
if image is not None and 'results' in dir():
    final_mask = results['Hybrid_Combined']
    final_analysis = analyze_detection(image, final_mask)
    
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    # Original
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title('Original Brain MRI', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    # Detection
    axes[1].imshow(image, cmap='gray')
    if final_analysis['detected']:
        axes[1].imshow(final_mask, cmap='Reds', alpha=0.5)
        if final_analysis['center']:
            row, col = final_analysis['center']
            axes[1].plot(col, row, 'g+', markersize=30, markeredgewidth=4)
    axes[1].set_title('Hybrid Detection Result', fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    # Results panel
    axes[2].axis('off')
    
    if final_analysis['detected']:
        if final_analysis['confidence'] > 0.65:
            status = '⚠️ TUMOR DETECTED'
            verdict = '🔴 HIGH CONFIDENCE'
            color = 'red'
        elif final_analysis['confidence'] > 0.4:
            status = '❓ POSSIBLE TUMOR'
            verdict = '🟡 MEDIUM CONFIDENCE'
            color = 'orange'
        else:
            status = '❓ SUSPICIOUS REGION'
            verdict = '🟠 LOW CONFIDENCE'
            color = 'orange'
    else:
        status = '✅ NO TUMOR DETECTED'
        verdict = '🟢 Brain appears normal'
        color = 'green'
    
    results_text = f"""
    
    {status}
    
    ───────────────────────────────
    
    Confidence: {final_analysis['confidence']*100:.1f}%
    
    Tumor Area: {final_analysis['area_percent']:.2f}%
    
    Detection Method: Hybrid
    (FFT + Blob Ensemble)
    
    ───────────────────────────────
    
    {verdict}
    """
    
    axes[2].text(0.05, 0.5, results_text, fontsize=13, verticalalignment='center',
                fontfamily='monospace', 
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    axes[2].set_title('Diagnosis', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print('\n' + '='*60)
    print(f'  FINAL VERDICT: {verdict}')
    print('='*60)
    
    if final_analysis['detected']:
        print(f"\nDetection Details:")
        print(f"  • Tumor location: row {final_analysis['center'][0]}, col {final_analysis['center'][1]}")
        print(f"  • Estimated size: {final_analysis['area_percent']:.2f}% of brain")
        print(f"  • Confidence level: {final_analysis['confidence']*100:.1f}%")
        print(f"\n⚕️ Recommendation: Further clinical review recommended")
    else:
        print(f"\n✓ No significant abnormalities detected")
        print(f"  Both FFT and spatial methods agree: normal brain tissue")
else:
    print('⚠️ Run detection first!')

---
## 📊 Validation (if ground truth available)

In [ ]:
if image is not None and mask is not None:
    detected = results['Hybrid_Combined']
    
    intersection = np.logical_and(detected, mask).sum()
    union = np.logical_or(detected, mask).sum()
    dice = 2 * intersection / (detected.sum() + mask.sum() + 1e-8)
    iou = intersection / (union + 1e-8)
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(image, cmap='gray')
    axes[0].imshow(mask, cmap='Greens', alpha=0.5)
    axes[0].set_title('Ground Truth', fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(image, cmap='gray')
    axes[1].imshow(detected, cmap='Reds', alpha=0.5)
    axes[1].set_title('Our Detection', fontweight='bold')
    axes[1].axis('off')
    
    overlap = np.zeros((*image.shape, 3))
    overlap[mask > 0] = [0, 1, 0]
    overlap[detected > 0] = [1, 0, 0]
    overlap[np.logical_and(mask, detected)] = [1, 1, 0]
    
    axes[2].imshow(image, cmap='gray')
    axes[2].imshow(overlap, alpha=0.6)
    axes[2].set_title(f'Overlap\nDice: {dice:.3f}, IoU: {iou:.3f}', fontweight='bold')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f'\n📊 Accuracy Metrics:')
    print(f'   Dice Score: {dice:.3f}')
    print(f'   IoU Score: {iou:.3f}')
else:
    print('ℹ️ No ground truth mask available')
    print('   Upload an image with corresponding mask for validation')

---
## 🔧 Adjust Sensitivity (Optional)

In [ ]:
if image is not None:
    print('Testing different sensitivity levels...\n')
    
    sensitivities = [0.3, 0.5, 0.7]
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    for idx, sens in enumerate(sensitivities):
        results_sens = hybrid_tumor_detection(image, sensitivity=sens)
        mask_sens = results_sens['Hybrid_Combined']
        analysis_sens = analyze_detection(image, mask_sens)
        
        axes[idx].imshow(image, cmap='gray')
        if analysis_sens['detected']:
            axes[idx].imshow(mask_sens, cmap='Reds', alpha=0.5)
        
        status = f"Detected: {analysis_sens['area_percent']:.1f}%" if analysis_sens['detected'] else "Not detected"
        axes[idx].set_title(f'Sensitivity: {sens}\n{status}', fontsize=12)
        axes[idx].axis('off')
    
    plt.suptitle('Sensitivity Analysis', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print('💡 Higher sensitivity = detects smaller/subtler tumors')
    print('   Lower sensitivity = more conservative, fewer false positives')
else:
    print('⚠️ No image loaded!')